In [1]:
#https://nbviewer.jupyter.org/url/nikhilvj.co.in/files/gtfsrt/locations.ipynb

In [241]:
from google.transit import gtfs_realtime_pb2
from google.protobuf.json_format import MessageToDict, MessageToJson
import urllib3
import requests
import time
import json
import pandas as pd
import datetime
from collections import OrderedDict
import numpy as np
from functools import reduce

In [242]:
stop_ids = pd.read_csv('https://openmobilitydata-data.s3-us-west-1.amazonaws.com/public/feeds/mta/79/20181221/original/stops.txt')


# FIX: CONVERT stop_name to stop_id

In [243]:
urls = ['http://datamine.mta.info/mta_esi.php?key=636e323a9180834b0811457aa7db81ce', 
'http://datamine.mta.info/mta_esi.php?key=636e323a9180834b0811457aa7db81ce&feed_id=16'
       ]

In [247]:
def get_feed(url):
    http = urllib3.PoolManager()
    feed = gtfs_realtime_pb2.FeedMessage()
    response = requests.get(url)
    feed.ParseFromString(response.content)
    dict_obj = MessageToDict(feed)
    return dict_obj

In [245]:
def get_time(stop, direction):
    #get feed
#     http = urllib3.PoolManager()
#     feed1 = gtfs_realtime_pb2.FeedMessage()
#     feed2 = gtfs_realtime_pb2.FeedMessage()
#     response1 = requests.get('http://datamine.mta.info/mta_esi.php?key=636e323a9180834b0811457aa7db81ce')
#     response2 = requests.get('http://datamine.mta.info/mta_esi.php?key=636e323a9180834b0811457aa7db81ce&feed_id=16')
#     feed1.ParseFromString(response1.content)
#     feed2.ParseFromString(response2.content)
#     dict_obj1 = MessageToDict(feed1)
#     dict_obj2 = MessageToDict(feed2)
    dicts = [get_feed(url) for url in urls]
    dict_obj = reduce(lambda a,b: a+b, dicts)
    collector = []
    #turn stop into stop id
    
    stop_id = stop_ids.loc[(stop_ids['stop_name']==stop), 'stop_id'].tolist()

    # turn feed dict object into DataFrame
    for block in dict_obj:

        row = OrderedDict()
        try:
            row['id'] = block['id']
            row['tripId'] = block['tripUpdate']['trip'].get('tripId','')
            row['routeId'] = block['tripUpdate']['trip'].get('routeId','')
            for i, stop in enumerate(block['tripUpdate']['stopTimeUpdate']):
                minutes = round((int(stop['arrival'].get('time','')) - int(time.time()))/60)
                row[i] = (stop['stopId'], minutes)

            collector.append(row)
        except:
            pass
        
        df = pd.DataFrame(collector)
        
    # print out timetables for given stop_id
    for i, row in df.iterrows():
        for  j in row[3:]:
            if type(j) == tuple:
                if j[0] in stop_id and j[0].endswith(direction):
                    print(row.routeId + ' arriving in ' + str(j[1]) + ' minutes', row.tripId)
 

In [255]:
get_feed(urls[1])

{'header': {'gtfsRealtimeVersion': '1.0', 'timestamp': '1577482876'},
 'entity': [{'id': '000001N',
   'tripUpdate': {'trip': {'tripId': '093596_N..S',
     'startTime': '15:35:58',
     'startDate': '20191227',
     'routeId': 'N'},
    'stopTimeUpdate': [{'arrival': {'time': '1577482929'},
      'departure': {'time': '1577482929'},
      'stopId': 'N08S'},
     {'arrival': {'time': '1577483190'},
      'departure': {'time': '1577483190'},
      'stopId': 'N09S'},
     {'arrival': {'time': '1577483250'},
      'departure': {'time': '1577483250'},
      'stopId': 'N10S'}]}},
  {'id': '000002N',
   'vehicle': {'trip': {'tripId': '093596_N..S',
     'startTime': '15:35:58',
     'startDate': '20191227',
     'routeId': 'N'},
    'currentStopSequence': 24,
    'stopId': 'N07'}},
  {'id': '000003N',
   'tripUpdate': {'trip': {'tripId': '094200_N..N',
     'startTime': '15:42:00',
     'startDate': '20191227',
     'routeId': 'N'},
    'stopTimeUpdate': [{'arrival': {'time': '1577482861'},


In [246]:
get_time('Atlantic Av - Barclays Ctr', 'N')

/Users/aarongittleman/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: Unexpected end-group tag: Not all data was converted
  """


KeyError: 'entity'

In [233]:
len(x)

635

In [196]:
x.items()

dict_items([('header', {'gtfsRealtimeVersion': '1.0', 'timestamp': '1577478179'}), ('entity', [{'id': '000001N', 'tripUpdate': {'trip': {'tripId': '086258_N..N', 'startTime': '14:22:35', 'startDate': '20191227', 'routeId': 'N'}, 'stopTimeUpdate': [{'arrival': {'time': '1577478172'}, 'departure': {'time': '1577478172'}, 'stopId': 'R08N'}, {'arrival': {'time': '1577478237'}, 'departure': {'time': '1577478237'}, 'stopId': 'R06N'}, {'arrival': {'time': '1577478327'}, 'departure': {'time': '1577478327'}, 'stopId': 'R05N'}, {'arrival': {'time': '1577478447'}, 'departure': {'time': '1577478447'}, 'stopId': 'R04N'}, {'arrival': {'time': '1577478507'}, 'departure': {'time': '1577478507'}, 'stopId': 'R03N'}, {'arrival': {'time': '1577478627'}, 'departure': {'time': '1577478627'}, 'stopId': 'R01N'}]}}, {'id': '000002N', 'vehicle': {'trip': {'tripId': '086258_N..N', 'startTime': '14:22:35', 'startDate': '20191227', 'routeId': 'N'}, 'currentStopSequence': 22, 'timestamp': '1577478172', 'stopId': 'R

In [158]:
get_time('Nevins St', 'N')

{'header': {'gtfsRealtimeVersion': '1.0', 'timestamp': '1577477648'},
 'entity': [{'id': '000001N',
   'tripUpdate': {'trip': {'tripId': '084253_N..N',
     'startTime': '14:02:32',
     'startDate': '20191227',
     'routeId': 'N'}}},
  {'id': '000002N',
   'vehicle': {'trip': {'tripId': '084253_N..N',
     'startTime': '14:02:32',
     'startDate': '20191227',
     'routeId': 'N'},
    'currentStopSequence': 27,
    'stopId': 'R01'}},
  {'id': '000003N',
   'tripUpdate': {'trip': {'tripId': '085265_N..N',
     'startTime': '14:12:39',
     'startDate': '20191227',
     'routeId': 'N'},
    'stopTimeUpdate': [{'arrival': {'time': '1577477632'},
      'departure': {'time': '1577477632'},
      'stopId': 'R08N'},
     {'arrival': {'time': '1577477697'},
      'departure': {'time': '1577477697'},
      'stopId': 'R06N'},
     {'arrival': {'time': '1577477787'},
      'departure': {'time': '1577477787'},
      'stopId': 'R05N'},
     {'arrival': {'time': '1577477907'},
      'departure': 

In [40]:
def get_time2(stop, direction):
    return get_time(stop, direction)

In [34]:
get_time2('Atlantic Av - Barclays Ctr', 'N')

4 arriving in 1 minutes 137000_4..N01R


In [26]:
def get_update(stop, direction):
    http = urllib3.PoolManager()
    feed = gtfs_realtime_pb2.FeedMessage()
    response = requests.get('http://datamine.mta.info/mta_esi.php?key=636e323a9180834b0811457aa7db81ce')
    feed.ParseFromString(response.content)
    dict_obj = MessageToDict(feed)
    collector = []
    
    for block in dict_obj['entity']:
        
        row = OrderedDict()
        try:
            row['id'] = block['id']

            row['trip_id'] = block['vehicle']['trip'].get('tripId','')
            row['route_id'] = block['vehicle']['trip'].get('routeId','')
            row['direction_id'] = block['vehicle']['trip'].get('directionId','')
            row['schedule_relationship'] = block['vehicle'].get('currentStatus','')

    
            row['start_date'] = block['vehicle']['trip'].get('startDate','')
            row['current_stop_sequence'] = block['vehicle'].get('currentStopSequence','')
            row['current_status'] = block['vehicle'].get('currentStatus','')
            row['timestamp'] = block['vehicle'].get('timestamp','')
            row['stop_id'] = block['vehicle'].get('stopId','')
            collector.append(row)
        except:
            pass

    df = pd.DataFrame(collector)
    
    feedtime = int(dict_obj['header']['timestamp'])
    #print('Feed timestamp:',feedtime,',', datetime.datetime.fromtimestamp(feedtime))
    df['feed time'] = datetime.datetime.fromtimestamp(feedtime)
    
    df = pd.merge(df, stop_ids[['stop_id', 'stop_name']],  on='stop_id', how='outer')
    
    stop_id = [x for x in df.loc[(df['stop_name'] == stop),'stop_id'].tolist() if (x[-1] == direction and x[0] != 'L')][0][:-1]
    
    #stops = []
    #if direction == 'N':
    #    stops.extend(range(int(stop_id), int(stop_id)+10))
    #elif direction == 'S':
    #    stops.extend(range(int(stop_id)-10 ,int(stop_id)))
    #   
    #stops = [str(x)+direction for x in stops]
    
    print(df)
    
   # print(df[((df.stop_id == stops[0]) |
   #(df.stop_id == stops[1]) |
   #(df.stop_id == stops[2])  |
   #(df.stop_id == stops[3]) |
   #(df.stop_id == stops[4]) |
   #(df.stop_id == stops[5]) |
   #(df.stop_id == stops[6]))][['current_status','route_id','stop_name', 'stop_id', 'trip_id']])
    
    return df, dict_obj['entity']

In [12]:
[x for x in df.loc[(df['stop_name'] == 'DeKalb Av'),'stop_id'].tolist() if (x[-1] == 'N' and x[0] != 'L')]

['R30N']

In [27]:
df, dict_ = get_update('Atlantic Av - Barclays Ctr', 'N')

          id         trip_id route_id direction_id schedule_relationship  \
0     000002  076350_1..S03R        1                      IN_TRANSIT_TO   
1     000004  076550_1..N03R        1                         STOPPED_AT   
2     000006  076950_1..S03R        1                         STOPPED_AT   
3     000008  077150_1..N03R        1                      IN_TRANSIT_TO   
4     000010  077550_1..S03R        1                         STOPPED_AT   
5     000012  077750_1..N03R        1                         STOPPED_AT   
6     000014  078150_1..S03R        1                         STOPPED_AT   
7     000016  078350_1..N03R        1                         STOPPED_AT   
8     000018  078750_1..S03R        1                         STOPPED_AT   
9     000020  078950_1..N03R        1                      IN_TRANSIT_TO   
10    000022  079350_1..S03R        1                         STOPPED_AT   
11    000024  079550_1..N03R        1                      IN_TRANSIT_TO   
12    000026

In [28]:
dict_

[{'id': '000001',
  'tripUpdate': {'trip': {'tripId': '076350_1..S03R',
    'startDate': '20191031',
    'routeId': '1'},
   'stopTimeUpdate': [{'arrival': {'time': '1572543746'}, 'stopId': '142S'}]}},
 {'id': '000002',
  'vehicle': {'trip': {'tripId': '076350_1..S03R',
    'startDate': '20191031',
    'routeId': '1'},
   'currentStopSequence': 38,
   'currentStatus': 'IN_TRANSIT_TO',
   'timestamp': '1572543608',
   'stopId': '142S'}},
 {'id': '000003',
  'tripUpdate': {'trip': {'tripId': '076550_1..N03R',
    'startDate': '20191031',
    'routeId': '1'},
   'stopTimeUpdate': [{'arrival': {'time': '1572543549'},
     'departure': {'time': '1572543699'},
     'stopId': '103N'},
    {'arrival': {'time': '1572543789'}, 'stopId': '101N'}]}},
 {'id': '000004',
  'vehicle': {'trip': {'tripId': '076550_1..N03R',
    'startDate': '20191031',
    'routeId': '1'},
   'currentStopSequence': 37,
   'currentStatus': 'STOPPED_AT',
   'timestamp': '1572543549',
   'stopId': '103N'}},
 {'id': '000005

In [16]:
[x for x in df.loc[(df['stop_name'] == '14 St'),'stop_id'].tolist()]

['132S', '132S', '132N']

In [75]:
df.loc[(df['stop_name'] == '14 St'),'stop_id'].tolist()

['132N', '132S']

In [22]:
while True:
    #try:
    get_time('Nevins St','N')
    get_update('Nevins St','N')
   # except:
    #    continue
    time.sleep(10)

2 arriving in 1 minutes 133500_2..N08X008
2 arriving in 16 minutes 135000_2..N08X008
4 arriving in 7 minutes 133650_4..N01R
4 arriving in 17 minutes 134650_4..N01R
    current_status route_id                       stop_name stop_id  \
29   IN_TRANSIT_TO        2                       Nevins St    234N   
48      STOPPED_AT        4  Eastern Pkwy - Brooklyn Museum    238N   
250            NaN      NaN      Atlantic Av - Barclays Ctr    235N   
253            NaN      NaN                       Bergen St    236N   
255            NaN      NaN                Grand Army Plaza    237N   
260            NaN      NaN                     Franklin Av    239N   

               trip_id  
29   133500_2..N08X008  
48      133650_4..N01R  
250                NaN  
253                NaN  
255                NaN  
260                NaN  
2 arriving in 0 minutes 133500_2..N08X008
2 arriving in 16 minutes 135000_2..N08X008
4 arriving in 7 minutes 133650_4..N01R
4 arriving in 17 minutes 134650_4..N01R

2 arriving in 14 minutes 135000_2..N08X008
4 arriving in 5 minutes 133650_4..N01R
4 arriving in 15 minutes 134650_4..N01R
    current_status route_id                       stop_name stop_id  \
48      STOPPED_AT        4                Grand Army Plaza    237N   
249            NaN      NaN                       Nevins St    234N   
252            NaN      NaN      Atlantic Av - Barclays Ctr    235N   
255            NaN      NaN                       Bergen St    236N   
258            NaN      NaN  Eastern Pkwy - Brooklyn Museum    238N   
261            NaN      NaN                     Franklin Av    239N   

            trip_id  
48   133650_4..N01R  
249             NaN  
252             NaN  
255             NaN  
258             NaN  
261             NaN  
2 arriving in 14 minutes 135000_2..N08X008
4 arriving in 5 minutes 133650_4..N01R
4 arriving in 15 minutes 134650_4..N01R
    current_status route_id                       stop_name stop_id  \
48      STOPPED_AT        4      

2 arriving in 12 minutes 135000_2..N08X008
4 arriving in 3 minutes 133650_4..N01R
4 arriving in 12 minutes 134650_4..N01R
4 arriving in 24 minutes 135700_4..N01R
    current_status route_id                       stop_name stop_id  \
42      STOPPED_AT        4                       Bergen St    236N   
247            NaN      NaN                       Nevins St    234N   
250            NaN      NaN      Atlantic Av - Barclays Ctr    235N   
255            NaN      NaN                Grand Army Plaza    237N   
257            NaN      NaN  Eastern Pkwy - Brooklyn Museum    238N   
259            NaN      NaN                     Franklin Av    239N   

            trip_id  
42   133650_4..N01R  
247             NaN  
250             NaN  
255             NaN  
257             NaN  
259             NaN  
2 arriving in 12 minutes 135000_2..N08X008
4 arriving in 3 minutes 133650_4..N01R
4 arriving in 12 minutes 134650_4..N01R
4 arriving in 24 minutes 135700_4..N01R
    current_status route

    current_status route_id                       stop_name stop_id  \
45      STOPPED_AT        4      Atlantic Av - Barclays Ctr    235N   
248            NaN      NaN                       Nevins St    234N   
253            NaN      NaN                       Bergen St    236N   
256            NaN      NaN                Grand Army Plaza    237N   
259            NaN      NaN  Eastern Pkwy - Brooklyn Museum    238N   
261            NaN      NaN                     Franklin Av    239N   

            trip_id  
45   133650_4..N01R  
248             NaN  
253             NaN  
256             NaN  
259             NaN  
261             NaN  
2 arriving in 11 minutes 135000_2..N08X008
4 arriving in 1 minutes 133650_4..N01R
4 arriving in 10 minutes 134650_4..N01R
4 arriving in 23 minutes 135700_4..N01R
    current_status route_id                       stop_name stop_id  \
44      STOPPED_AT        4      Atlantic Av - Barclays Ctr    235N   
249            NaN      NaN                 

KeyboardInterrupt: 

In [66]:
stop_ids.groupby('stop_name')['stop_id'].nunique()['Atlantic Av - Barclays Ctr']

9

# SCRATCH

In [3]:
stop_ids = pd.read_csv('stop_ids.csv')

In [4]:
stop_ids.columns

Index(['Unnamed: 0', 'stop_id', 'stop_code', 'stop_name', 'stop_desc',
       'stop_lat', 'stop_lon', 'zone_id', 'stop_url', 'location_type',
       'parent_station'],
      dtype='object')

In [12]:
 set(stop_ids['stop_name'].to_list())

{'1 Av',
 '103 St',
 '103 St - Corona Plaza',
 '104 St',
 '110 St',
 '111 St',
 '116 St',
 '116 St - Columbia University',
 '121 St',
 '125 St',
 '135 St',
 '137 St - City College',
 '138 St - Grand Concourse',
 '14 St',
 '14 St - Union Sq',
 '145 St',
 '149 St - Grand Concourse',
 '15 St - Prospect Park',
 '155 St',
 '157 St',
 '161 St - Yankee Stadium',
 '163 St - Amsterdam Av',
 '167 St',
 '168 St',
 '168 St - Washington Hts',
 '169 St',
 '170 St',
 '174 St',
 '174-175 Sts',
 '175 St',
 '176 St',
 '18 Av',
 '18 St',
 '181 St',
 '182-183 Sts',
 '183 St',
 '190 St',
 '191 St',
 '2 Av',
 '20 Av',
 '207 St',
 '21 St',
 '21 St - Queensbridge',
 '215 St',
 '219 St',
 '225 St',
 '23 St',
 '231 St',
 '233 St',
 '238 St',
 '25 Av',
 '25 St',
 '28 St',
 '3 Av',
 '3 Av - 138 St',
 '3 Av - 149 St',
 '30 Av',
 '33 St',
 '34 St - 11 Av',
 '34 St - Herald Sq',
 '34 St - Penn Station',
 '36 Av',
 '36 St',
 '39 Av',
 '4 Av',
 '40 St',
 '42 St - Bryant Pk',
 '42 St - Port Authority Bus Terminal',
 '4

In [151]:
a = {'bill': 4, 'kate': 5}
b = {'carl': 6, 'bob': 4}

In [152]:
a.update(b)

In [153]:
a.keys()

dict_keys(['bill', 'kate', 'carl', 'bob'])